Team members and primary contributions:
- Aadi Kothari (aadi@mit.edu): RRT/RRT* and helping code/run linear programming.
- Anup Sreekumar (anupsk@mit.edu): Linear Programming
- Tom Stuart (trstuart@mit.edu): Fault diagnoses

Note: Due to collective nature of the problem, teammates have worked together on some aspects of each subgroup.

Note: you may need to install the following packages to run this notebook:

1. Shapely -- see https://pypi.org/project/Shapely/ for installation instructions.
2. Descartes -- see https://docs.descarteslabs.com/installation.html for installation instructions.
3. Pulp -- see https://pypi.org/project/PuLP/ for installation instructions
4. Time -- see https://pypi.org/project/python-time/
5. Copy -- see https://docs.python.org/3/library/copy.html


In [ ]:
# Useful imports
from __future__ import division
%load_ext autoreload
%autoreload 2
%matplotlib inline
import yaml
import numpy as np
from matplotlib import pyplot as plt
from shapely.geometry import Point, Polygon, LineString, box
from environment import Environment, plot_environment, plot_line, plot_poly, random_environment
import random as random

# Aerial Delivery and Research Support

## **The Problem** 

You lead the operations team for a group of researchers operating at remote locations throughout the Denali Wilderness. From your base camp in Denali National Park, you are responsible for delivering food, medical supplies, and equipment to the research teams located at 4 different locations throughout the wilderness--they endeavor to work at these locations throughout the summer. You are also responsible for supporting their research requirements to include maintenance of scientific equipment.

*Add in some cool pictures*

The terrain in Denali is extremely rugged and consists of a mix of taiga forests, high tundra, glaciers, and mountains. The research teams operate in locations unreachable by road and the scientists get in place by hiking in and out of their working locations. Currently supplies are delivered in the same way. This makes maintaining a continuous research presence extremely difficult as weather events, unexpected supply shortages, and other operational issues often lead to dangerously low provisions at the research sites--forcing the scientists to return intermittently to ensure the safety of the team.

Fortunately your team has just come into possession of 7 drones who are capable of delivering supplies to the teams. Your task is to:

<ol>
    <li><b> Plan routes between all sites and the base camp avoiding designated no-fly areas.</b></li>
    <li><b> Develop a supply delivery schedule based upon your planned routes, your drone delivery capabilities, and research team needs.</b></li>
    <li><b> Develop a method to diagnose scientific equipment faults and issues to determine when equipment supplies need to be delivered to a team.<b/></li>
</ol>

## **Route Planning with RRT and RRT*** 

Your drones can overfly most of terrain between you and the research teams but must take care to avoid areas designated noise sensitive due to wildlife, ecologically sensitive areas where it is deemed completely unnacceptable for a drone to crash, and campsites in the national park near your launch location. All of these areas were designated no-fly areas by the Bureau of Land Management and the National Park Service. Finally, some high altitude areas cannot be overflown due to drone performance limitations.

Your team has developed a map depicting these no-fly areas, your base camp, and the research sites. Anything outside the map boundaries is also considered a no-fly area. Based on the ecological sensitivity of Denali (and the importance that the National Park Service puts on keeping the wilderness as pristine as possible!) there are <ins> a lot </ins> of no-fly areas. Your drones can be relied upon to stay within 100 meters of any planned path (this navigation error will be exceeded with a probability of 10<sup>-5</sup> in any given flight-hour so it's acceptable use this number for path planning!).

Due to the high latitude of your operations, your team uses a grid system to specify camp locations referenced to True North. For the remote camps, you are provided with a landing zone (LZ) geometry. Each unit digit on the grid system represents one kilometer. Hence, location (-4,-2) is one kilometer *true* south of position (-3,-2).

Based on this grid system, the relevant locations are:

<ul class="dashed">
  <li>The base camp drone launch site is located at position (-2,-2)</li>
  <li>The site 1 LZ bounds are (-4.5,3.6), (-4.5,4), (-4.2,4),(-4.2,3.6).</li>
  <li>The site 2 LZ bounds are (9.7,0), (9.7,0.4), (10,0.4),(10,0).</li>
   <li>The site 3 LZ bounds are (12.7,3.6), (12.7,4), (13,4),(13,3.6).</li>
   <li>The site 4 LZ bounds are (13.6,-3.2), (13.6,-3.5), (14,-3.5),(14,-3.2).</li>
</ul>

**The code block below shows the map of the no-fly areas (in blue), the base camp (in magenta), and the LZs (in green).**

In [ ]:
# Plot the environment 
env = Environment('Denali_650.yaml')
ax = plot_environment(env)

# Base camp location
radius = 0.1
start_pose = (-2,-2)
start_point = Point(start_pose[0],start_pose[1])
start_ball = start_point.buffer(radius)

# Camp locations
LZ1 = Polygon([(-4.5,3.6), (-4.5,4), (-4.2,4),(-4.2,3.6)])
LZ2 = Polygon([(9.7,0), (9.7,0.4), (10,0.4),(10,0)])
LZ3 = Polygon([(12.7,3.6), (12.7,4), (13,4),(13,3.6)])
LZ4 = Polygon([(13.6,-3.2), (13.6,-3.5), (14,-3.5),(14,-3.2)])

LZs = (LZ1, LZ2, LZ3, LZ4)


# Plot base camp in magenta
plot_poly(ax, start_ball,'magenta')
# Plot research camps in green
for LZ in LZs:
    plot_poly(ax, LZ,'green')

<ins>**Rapidly Exploring Random Trees (RRT)**</ins>

Looking at the map, you are a bit baffled in how you can make sure you find the best routes through this maze. Fortunately, you took 16.413 at MIT and recall learning about sampling-based planning methods--in particular Rapidly Exploring Random Trees (RRT). You decide to first tackle this problem with RRT. We chose RRT over graph search primarily due to the curse of dimensionality as highlighted in the course. For a better path, we would like to discretize our map further which exponentially increases the node size in case of a graph search algoithm. Further, we can also incoporate our agent's dynamics using RRT since we simulate forward (much harder to implement in probabilistic roadmap planning).

**Academic description of RRT**
In RRT (Rapidly Exploring Random Trees), the search tree is incrementally generated using samples from search space in a random fashion. In the iterative process:
<li> A random coordinate is chosen in free space </li>
<li> Find the nearest vertex exisiting in the tree to that coordinate</li>
<li> Steer that vertex in the direction of the coordinate by a depth d</li>
<li> If the new vertex is obstacle free, add it to our tree.</li>
<li> Repeat the above steps until the vertex lands in the goal region</li>
RRT is probabilistically complete which means that it will find a solution in case the solution exists (though it may take a long time to run). A few downsides of RRT are that it usually returns a suboptimal path and also fails to check if a path does not exist (i.e. the algorithm will run forever in case the path between start and goal location does not exist).

**RRT Demo going from Base Camp --> LZ1, discuss how non-optimal**

Taking a look at the produced route, you can tell by inspection that the route is clearly not optimal. Thankfully, you remember that there is an optimal version of RRT, RRT* that you should be able to implement for this problem.

<b> Academic description of RRT* </b>
RRT* is an optimized version of RRT, and is asymptotically optimal. In an ideal case when number of nodes tend to infinity, RRT* yields the shortest path (optimal path) to the goal by keeping track of the distance each vertex has travelled relative to the parent vertex. Another variation is that after a vertex is added to the lowest cost neighbor, the tree is rewired to the new vertex in order to decrease individual cost. In the the iterative process:
<li> A random coordinate is chosen in free space that is not in any obstacle</li>
<li> Find the nearest node existing in the tree to that coordinate</li>
<li> Store the cost i.e. the distance between the nearest node to the coordinate </li>
<li> Examine neighbors within a specified radius of the new node and updates costs for them if combined cost with the new node turns out to be less than the existing cost of that neighbor.</li>
<li> Update parent of that graph and the node accordingly.</li>
<li> Repeat the above steps until the vertex lands in the goal region</li>
Note: Instead of the above terminating condition, we can run the code for more iterations to find a better (more optimal) path. Also, if we have the agent dynamics, we can find the "nearest node" that is closest to the possible achievable states of the agent.

<ins>**Motion planning solution summarazied**</ins>
**Input** : The map of the delivery region and the coordinates of the basecamp and the five destinations <br>
**Output** : The distance matrix between basecamp and destinations <br>
**Method** : Run RRT/RRT* among all points (basecamp and destinations) and populate with the shortest distance <br>

In [ ]:
# Define the maps containing start and goal nodes along with obstacles

# Use RRT/RRT* to calculate the optimal distance matrix coresponding to the distances between basecamp and destinations


# **Mixed Integer Programming to determine Drone Routing & Drop Point Assignemnt to each drone**

Through the RRT/RRT* module we will populate shortest path from basecamp to each drop points and from each drop point to every other drop point. The output will be a 5x5 matrix with [0,0] representing the basecamp and the other pair representing the distances. <br>
<br>Once this data is populated, we used a MIP (linear) model to determine the following objectives : <br>
1. How many drones and which tye of drone to be used<br>
2. The route to be taken by each drone with drop point assignment in the route<br>
3. The percentage of the load of drop point to be taken by each drone (if a drop point is serviced by multiple drones)<br>

The overall model - Objective function, Parameters, and Constraints - is explained below both in mathematical formulations and in python code. 

## **A. Concept Introduction** <br>
MIP is a method employed to maximize or minimize an objective function (represented by a mathemetical equation) against constraints which are also represented in mathematical formulation. In our model we are using linear relationships between variables both in the objective fucntion and in the cosntratins. <br>

The three major components of a MIP is 
<br><br>
(i) Objective function : What the model should maximize or minimize . This is usually modelled mathematically using the decision variables and constants. Refer Mathematical model, in which we explain how we are defining the obejctive fucntion for our model . 
<br>
(ii) Decision Variables : These are the variables that the MIP model optimizes ( or iterates with ) to achieve the objective function <br>
(iii) Constraints : These are the relationships or bounds for the decision variables that the model needs to ensure is within the limits . For our models, these relations are modelled in linear relationship. 
<br><br>
The standrad form of a MIP is shown below <br>
Reference : https://www.gurobi.com/resource/mip-basics/ <br>
<br>
Find a vector  **X**        <-- Decision Variable <br>
that maximizes $  C^T X $   <-- Objective Functiobn <br>
subject to     $ AX = B $  <-- Linear constraints  <br>
and            $ L <= X <= U $   <-- L,U are upper and lower Bound constraints <br>
and            Some or all X will take integer values


## **B. MIP Mathematical Model for Aerial delivery system**

### **1. Data Set used in the model** : 
1. N_drops: Number of drop points ( 4 drop point and 1 basecamp) 
2. N_drones : Number of drones in system ( 3 ) 
3. N_property : Property of demand and drones ( as Weight (Kgs) and Volume ( Meter cube ~ $m^3$ )
4. N_trips : Number of trips by the drone ( Maximum is set to 5 ) 
   Trip is a truck round route from basecamp to drop points and back to warehouse

### **2. Indexes used in the model** :
1. i,j,h refers to set drop points from 1 to N_drops ( i=j=h= 0 warehouse )
2. t refers to set drones from 1 to N_drones
3. d in set trips from 1 to N_trips
4. p in property from 1 to N_property
(Note: In python implemenataion the start for index is 0 and ends at N-1)

### **3. Parameters or Input to the Program** : <br>
1. Distance matrix between each point to every other point from RRT/RRT* <br>
    Represented as $D_{ij}$ (i=j=0 refers to the basecamp)<br>
    (The distance $D_{ij}$ is kept to very huge along the diagnol . Please refer special notes under constraints )<br><br>
2. Fixed cost of using the drone in USD / Trip  <br>
    Representated as $FCT$
    This represents the investment costs, or rental or lease costs.<br> 
    (Refer the values in the python code section. The values have been taken from https://filmora.wondershare.com/drones/top-heavy-lift-drones.html )<br><br>
3. Variable cost of using the drone in USD/ Km for each Trip <br>
    Represented as $VC_t$<br>
    This represents the cost of maintaining the drone, the charging costs and distance related costs<br>
    (Refer the values in the python code section. The values have been taken from https://filmora.wondershare.com/drones/top-heavy-lift-drones.html )<br><br>
4. Averge speed of the drone in km/hr <br>
    Represented as $S_t$<br>
    We are assuming Speed to be 50 Kmph <br><br>
5. Demand of each drop point in terms of weight (tons) and volume (cubic meter)<br>
    Represented as $Demand_{ip}$<br>
    We have made assumptions for each drop point in terms of weight (tons) and volume (cubic meter)<br><br>
6. Capacity of drone in terms of weight (tons) and volume (cubic meter)<br>
    Represented as $Capacity_{tp}$<br>
    We took reference on current drone capability in terms of weight (tons) and volume (cubic meter)<br><br>
7. Eficiency of drone could be used ( % of weight or volume capacity could be used to transport)<br>
    Represented as $Eff$<br>
    Represented as Percentage(%)<br><br>
8. Max Range that a drone could travel ( in Kms )<br>
    Represented as $maxRange$<br>
    This is derived from the average speed (km per hour) and time limits.<br>
    The values have been taken from https://filmora.wondershare.com/drones/top-heavy-lift-drones.html )<br><br>
<br>
(All the values for these variables can be seen in  python code)

### **4. Decisions Variable** : <br>
For the model we are defining four variables and the domain values <br>
1. $X_{ijdt}$ refers to the percentage of demand moved from basecamp to each drop point in a trip d of drone t <Br>
    $ X_{ijdt} $  defined as a continuous variable between 0 and 1 <br><br> 
2. $Y_{dt}$ represents to whether a drone d is used in a trip t ( If $Y_{dt}$ = 1 means drone is used, else 0) <br>
    $Y_{dt}$ defined as a binary integer <br><br>
3. $Z_{ijdt}$ represents a binary integer. $Z_{ijdt}$ = 1 if some load is moved from Supplier i to Supplier j in drone d in trip t<br>
    $Z_{ijdt}$ defined as a binary integer <br><br>  
4. $R_{idt}$ : Rank of supplier i to be picked up on truck t on trip d. The concept of rank will be explained in the constraints in detail ( refer constraint number 6) <br>
    $R_{idt}$ defined as a integer <br><br>

    For ease of mathematical modelling, the fixed cost portion in the objective function is modelled as a constraint<br>
5. $FC$ Variable to calculate the fixed costs . The formulation is provided in the constratint 11 <br><br>

### **5. Objective function** :
The objective function is to **Minimize the total cost of delivery of supplies**. <br><br> 
Out intention is to have 2 components to total cost. <br>

Fixed cost : Fixed cost of using the drone (Investment cost or Lease/rental charges defined as USD per trip for each drone) <br>
    Fixed costs (FC) = $ \sum\limits_{d=1}^{NTrips}\sum\limits_{t=1}^{Ndrones} ( Y_{dt} * FCT )  $

Then we modelled the Total cost as : <br>
    
    Total Costs = Minimize 

$ (\sum\limits_{i=1}^{NDrops}\sum\limits_{j=1}^{NDrops}\sum\limits_{d=1}^{NTrips}\sum\limits_{t=1}^{Ndrones}
        Z_{ijdt} * D_{ij} * VC_t )  + FC$
<br> (Note that the last term FC is outside the first summaton group)

### **6. Constraints to the model** :
<br>    
0. Constraint 0  :<br><br>
    We do not want any movement from i to j, if i == j . These $X_{ijdt}$ should be forced to be 0 across all d and t <br>
    $X_{ijdt}$ = 0 $\forall d,t$ and $\forall i==j$
<br><br>   
1. Constraint 1 :<br><br>
    We want $Y_{dt}$ to be 1 , if the drone t is used in trip t and this depends on $X_{ijdt}$.
    If $X_{ijdt}$ >0 for a patricular drone t in trip t, then the corresponding $Y_{dt}$ <br>
    This is modelled as below : <br><br>
    $Y_{dt}$ >= $X_{ijdt}$ $\forall i,j,d,t$
 
    Note that if is $X_{ijdt}$ = 0, then the objective function will force $Y_{dt}$ to be 0 
<br><br> 
2. Constraint 2 :<br><br>
    We want $Z_{ijdt}$ to be 1, if any $X_{ijdt}$ is greater than 0 .<br>
    If $X_{ijdt}$ >0 for a patricular drone t in trip t, then the corresponding $Z_{ijdt}$ to be 1<br>
    Similar to constraint 1, if $X_{ijdt}$ = 0, the minimizing objective function forces $Z_{ijdt}$ to be 0<br>
    This is modelled as below : <br><br>
    $Z_{ijdt}$ >= $X_{ijdt}$ $\forall i,j,d,t$   
<br><br>    
3. Constraint 3 :<br><br> 
    We want all demand of each drop point to be >= 1 (Percentage >= 100) any of drones t in any of the drones t.<br>
    This is modelled as below : <br><br>
    $\sum\limits_{j=1}^{NDrops}\sum\limits_{d=1}^{NTrips}\sum\limits_{t=1}^{Ndrones} X_{ijdt} \geq$ 1 $\forall i  $
<br><br>     
4. Constraint 4 :<br><br> 
    The total load carried by the drone is less than the ton and volume capacity of drone t in all trips d.<br>
    We multiply the capacities of the drone d , with an "Eff" factor to factor in whether we can use the full capacuty or some portion of it ( eg 95% ) <br>
    This is modelled as below : <br><br>
    $\sum\limits_{i=1}^{NDrops} ( \sum\limits_{j=1}^{NDrops} X_{ijdt} * Demand_{ip} ) \leq$ $ Eff * Capacity_{tp} \forall d,t,p  $
<br><br>    
5. Constraint 5 :<br><br>
    We need to ensure that the drone does not fly beyond its rated range (in Kms) for the entire trip d for all drones d<br>
    This is modelled as below : <br><br>
    $\sum\limits_{i=1}^{NDrops}\sum\limits_{j=1}^{NDrops}\sum\limits_{d=1}^{NTrips} Z_{ijdt} * D_{ij} \leq maxRange_t$ $\forall t $
<br><br>     
6. Constraint 6 : <br><br>
    Sub Tour Eimination:<br> 
    A sub tour is a anormality that occurs in the transporatiaton problem <br> 
    Let us assume there is 1 basecamp (P) and 4 Suppliers (A,B,C,D) <br>
    Sub tour is a condition in which the drone will start from B go to C and then go back to C<br>
    We always want the drone to start from P and come back to P <br> 
    To ensure this we need to add an additional variable $R_{idt}$ <br>
    We will first add the rank of P to be 1 for all t and d. <br>
    The next supplier in the route will be assigned a rank higher than 1. <br>
    The idea is to give a rank to each supplier from where goods are picked up and ensure that no lower rank is picked up in the same trip ( not go back to same supplier ) and every new supplier to be picked up is given next higher rank. <br>
    Note: We need to use the Big M Method here to ignore this constraint when the $Z_{ijdt} = 0$. The big M is assigned N_drops<br>
    This is modelled as below in 2 lines : <br><br>
    (6.a) <br>
    $R_{idt} =1 $ $ \forall d,t$
    <br><br>
    (6.a) <br>
    $R_{jdt} \geq R_{jdt} + 1 - ( Ndrops * ( 1 - Z_{ijdt} ) ) $
    <br>
    The second term will become 0 if $Z_{ijdt}$ = 1, otherwise it does not matter what value $R_{jdt}$ takes when $Z_{ijdt}$ = 0
<br><br>     
7. Constraint 7 : <br><br>    
    As mentioned in costraint 6,we need to ensure that the drone leaves always from basecamp<br>
    This is modelled as below in 2 lines : <br><br>
    $\sum\limits_{i=1}^{NDrops} Z_{ijdt}$ = 1 $\forall$ $i=1,d,t$
<br><br>     
8. Constraint 8 : <br><br>    
    Balancing Constraints <br>
    We need to ensure that what comes into a drop point must leave from drop point also <br>
    This is only applicable to drop point and not the basecamp<br>
    This is modelled as below : <br><br>
    $\sum\limits_{i=1}^{NDrops} Z_{ihdt}$ = $\sum\limits_{j=1}^{NDrops} Z_{hjdt}$ $\forall$  $(i,j\geq2),d,t$
    
##### Special notes: 
1. In python implementation the start for index is 0 and ends at N-1
2. In the Distance i,j parameter data , the diagnol distance (between i and j when i == j is kept at arbitarily high. This is to avoid the $Z_{ijdt}$ from taking a value of 1 in that position.
3. The big M method used in constrain6 6 can be found here for a good explanation (http://www.universalteacherpublications.com/univ/ebooks/or/Ch3/mmethod.htm)

### **C. Python Code** 

#### **1. Packages used for Optimization**:

In [1]:
import numpy as np
from pulp import * # for optimization
import time
import copy

#### **2. Parameters, Dataset, Indexes used in the model** : 

In [2]:
#1. Distance matrix between each point to every other point from RRT/RRT* 
distance = [[0,419,310,25,49,26],
            [419,50000,592,445,483,413],
            [310,592,50000,335,358,288],
            [25,445,335,50000,30,51],
            [49,483,358,30,50000,75],
            [26,413,288,51,75,50000]] #50000 is done to minimize diagonola entries

#actual distance from RRT*
distance = [[ 0,6.0602571,10.10200439,6.18885202,6.53895829],
             [ 6.0602571,50000,7.09077131,4.42892364,6.15810891],
             [10.10200439,7.09077131,50000,3.99899854,13.09472191],
             [6.18885202,4.42892364,3.99899854,50000,9.84846108],
             [6.53895829,6.15810891,13.09472191,9.84846108,50000]]
#2. Fixed cost of using the drone in USD / Trip
FCT = 126 # this is the fixed cost of running the drone

#3. Variable cost of using the drone in USD/ Km for each Trip
VC = [2,1.5,1] # this is the variable cost of running the drone per km

#4. Averge speed of the drone in km/hr
Speed = 50 #Km.hr

#5. Demand of each drop point in terms of weight (tons) and volume (cubic meter)
demand = [[0,0],[25,0.03],[14,0.1],[7,0.04],[3,0.03]]     

#6. Capacity of drone in terms of weight (tons) and volume (cubic meter)
capacity = [[20,0.5],[10,0.3],[5,0.3]]

#7. Eficiency of drone could be used ( % of weight or volume capacity could be used to transport)
eff = 0.95 

#8. Max Range that a drone could travel ( in Kms )
max_range = [20,40,50]  # km is the max 

# Data Set used in the model
nDrops = 5 # 0...4
nTrips = 3 # 0 ... 4
nDrone = 3
nProp = 2

# Creating Indexes:
Drop = [i for i in range(nDrops)]
Trips = [i for i in range(nTrips)]
Drones = [i for i in range(nDrone)]
Property = [i for i in range(nProp)]

#### **3. Decisions Variables** : 

In [3]:
iter = [(i,j,d,t) for t in range(nDrone) for d in range(nTrips) for j in range(nDrops) for i in range(nDrops)]
# Decision variables
X_ijdt = LpVariable.dicts("X",(Drop,Drop, Trips, Drones),0,1)
Y_dt = LpVariable.dicts("Y",(Trips, Drones),0,1, cat = 'Binary')
Z_ijdt = LpVariable.dicts("Z",(Drop,Drop, Trips, Drones),0,1, cat = 'Binary')
R_idt = LpVariable.dicts("R",(Drop, Trips, Drones),0,None, cat = 'Integer')
FC = LpVariable("FC",0,None, cat = 'Integer')



#### **4. Objective Function** : 

In [4]:
# Objective funtion
B = LpProblem("Transportation_variable_cost_problem",LpMinimize)
B += lpSum((Z_ijdt[i][j][d][t]*distance[i][j]*VC[t] ) for (i,j,d,t) in iter ) +FC, "Total_Costs"

#### **5. Constraints** : 

In [5]:
#constraint 0
for i in Drop:
    for j in Drop:
        for t in Drones:
            for d in Trips:
                    if i == j:
                        B += X_ijdt[i][j][d][t] == 0 # Constraint 0
                        
# Constraints 1,2
for i in Drop:
    for j in Drop:
        for t in Drones:
            for d in Trips:
                    B += Y_dt[d][t] >= X_ijdt[i][j][d][t] # Constraint 1
                    B += Z_ijdt[i][j][d][t] >= X_ijdt[i][j][d][t] # Constraint 2
                     
#constraint 3
for i in Drop:
    B += lpSum([X_ijdt[i][j][d][t] for j in Drop for d in Trips for t in Drones ]) >= 1
    
#constraint 4
for d in Trips:
    for t in Drones:
        for p in Property:
            B += lpSum([ X_ijdt[i][j][d][t] * demand[i][p] for i in Drop for j in Drop  ] ) <= eff * capacity[t][p]
            
#constraint 5
for t in Drones:
    for d in Trips:
        B += lpSum([ Z_ijdt[i][j][d][t] * distance[i][j] for i in Drop for j in Drop ] ) <= max_range[t]

#constraint 6a
for i in Drop:
    for d in Trips:
        for t in Drones:
            for j in Drop:
                if j>=1:
                    B += R_idt[j][d][t] >= R_idt[i][d][t] + 1 - ( nDrops * ( 1- Z_ijdt[i][j][d][t]) )

#constraint 6b
for d in Trips:
    for t in Drones:
        B += R_idt[0][d][t] == 1 
        
#constraint 7
for d in Trips:
    for t in Drones:
        B += lpSum([ Z_ijdt[0][j][d][t] for j in Drop] ) == 1
        
#constraint 8
for d in Trips:
    for t in Drones:
        for h in Drop:
            if h==0:
                continue
            B += lpSum([ Z_ijdt[i][h][d][t] for i in Drop] ) == lpSum([ Z_ijdt[h][j][d][t] for j in Drop] ) 

#constraint 9
B += lpSum( Y_dt[d][t] * FCT for d in Trips for t in Drones  ) <= FC

In [6]:
distance = [[ 0,6.0602571,10.10200439,6.18885202,6.53895829],
             [ 6.0602571,50000,7.09077131,4.42892364,6.15810891],
             [10.10200439,7.09077131,50000,3.99899854,13.09472191],
             [6.18885202,4.42892364,3.99899854,50000,9.84846108],
             [6.53895829,6.15810891,13.09472191,9.84846108,50000]]

In [7]:
distance[0][4]

6.53895829

#### **6. Solving the Model** : 

In [8]:
start = time.time()

In [9]:
# Please note that it may take upto 10 minutes to run. 
B.solve()
finish = time.time()

#### **7. Model Status, Time Performance, and Output** : 

In [10]:
print("Status:", LpStatus[B.status])

Status: Optimal


In [11]:
print("Time (Seconds) to find a solution :",finish-start)

Time (Seconds) to find a solution : 350.8952724933624


In [12]:
# The optimised objective function value is printed to the screen    
print("Total Cost of Transportation = ", value(B.objective))

Total Cost of Transportation =  625.38073607


In [13]:

# Each of the variables is printed with it's resolved optimum value
for v in B.variables():
    print(v.name, "=", v.varValue)
# The optimised objective function value is printed to the screen    
print("Total Cost of Transportation = ", value(B.objective))

FC = 504.0
R_0_0_0 = 1.0
R_0_0_1 = 1.0
R_0_0_2 = 1.0
R_0_1_0 = 1.0
R_0_1_1 = 1.0
R_0_1_2 = 1.0
R_0_2_0 = 1.0
R_0_2_1 = 1.0
R_0_2_2 = 1.0
R_1_0_0 = 4.0
R_1_0_1 = 4.0
R_1_0_2 = 4.0
R_1_1_0 = 2.0
R_1_1_1 = 2.0
R_1_1_2 = 0.0
R_1_2_0 = 4.0
R_1_2_1 = 0.0
R_1_2_2 = 4.0
R_2_0_0 = 4.0
R_2_0_1 = 4.0
R_2_0_2 = 0.0
R_2_1_0 = 6.0
R_2_1_1 = 2.0
R_2_1_2 = 0.0
R_2_2_0 = 0.0
R_2_2_1 = 4.0
R_2_2_2 = 2.0
R_3_0_0 = 0.0
R_3_0_1 = 0.0
R_3_0_2 = 4.0
R_3_1_0 = 3.0
R_3_1_1 = 2.0
R_3_1_2 = 0.0
R_3_2_0 = 0.0
R_3_2_1 = 4.0
R_3_2_2 = 4.0
R_4_0_0 = 0.0
R_4_0_1 = 0.0
R_4_0_2 = 0.0
R_4_1_0 = 2.0
R_4_1_1 = 6.0
R_4_1_2 = 0.0
R_4_2_0 = 2.0
R_4_2_1 = 0.0
R_4_2_2 = 0.0
X_0_0_0_0 = 0.0
X_0_0_0_1 = 0.0
X_0_0_0_2 = 0.0
X_0_0_1_0 = 0.0
X_0_0_1_1 = 0.0
X_0_0_1_2 = 0.0
X_0_0_2_0 = 0.0
X_0_0_2_1 = 0.0
X_0_0_2_2 = 0.0
X_0_1_0_0 = 0.0
X_0_1_0_1 = 0.0
X_0_1_0_2 = 0.0
X_0_1_1_0 = 1.0
X_0_1_1_1 = 0.0
X_0_1_1_2 = 0.0
X_0_1_2_0 = 0.0
X_0_1_2_1 = 0.0
X_0_1_2_2 = 0.0
X_0_2_0_0 = 0.0
X_0_2_0_1 = 0.0
X_0_2_0_2 = 0.0
X_0_2_1_0 = 0.0
X_0_2_1

#### **8. Creating the Route Sequence from the Model** : 

In [14]:
# create  2 list, one for variable name and another for value
var = []
value = []
for v in B.variables():
    var.append(v.name)
    value.append(v.varValue)
#extracting only X variable from list of variables
check = 'X'
summ = [idx for idx in var if idx[0].lower() == check.lower()]
#print (summ)
#extracting X variabels and values into a dictioanry
res = {}
for i in range(len(var)):
        res[var[i]] = value[i]
#print (res)
finalx = {}
for i in range(len(summ)):
    finalx[summ[i]] = res[summ[i]]
#print (finalx)
#Removing all dictionary rows with value equal to 0
for key in list(finalx.keys()):  ## creates a list of all keys
    if finalx[key] == 0:
        del finalx[key]
#print(finalx)
#Taking out keys into separte list
R = []
for key in list(finalx.keys()):  ## creates a list of all keys
    R.append(key)
m=[]
n=[]
print(R)
for j in range(len(R)):
    n=[]
    for i in range(len(R[j])):
        #print(R[j][i])
        
        if(R[j][i] == "_"):
            continue
        n.append(R[j][i])
    m.append(n)
#print(m)
for i in m:
    #print (i)
    a = len(i)
    i.append(i[1]+i[2])
    i.append(i[3]+i[4])
tr = []
for i in m:
    #print (i)
    tr.append(i[6])
#print(tr)
tr1 = []
[tr1.append(x) for x in tr if x not in tr1]
#print(tr1)
f=[]
g=[]
for j in tr1:
    #print (j)
    f=[]
    for i in range(len(m)):
        if (m[i][6] == j):
               f.append(m[i][5])
    g.append(f)    
#print(g)
def sequence (q,start):
    for i in range(len(q)):
        if q[i][0]==start: 
            start = q[i][1]
            return q[i]
h=[]
for j in range(len(g)):
    q = g[j]
    #print(q)
    a=[]
    start ='0'

    for i in range(len(q)):
        y = sequence(q,start)
        start = y[1]
        a.append(y)
        if start == 0 :
            break
    h.append(a) 
#print(h)

['X_0_1_1_0', 'X_0_2_1_1', 'X_0_2_2_2', 'X_0_4_2_0', 'X_1_0_2_0', 'X_1_3_1_0', 'X_2_0_1_1', 'X_2_0_2_2', 'X_3_0_1_0', 'X_4_1_2_0']


In [15]:
for i in range(len(tr1)):
    print("Trip #",i,"- Drone:",tr1[i][1],"( Trip-",tr1[i][0],") Drop Sequence :",h[i])

print("Interpretation:")
print("If output is ['03', '31', '10'] : It means it will move from 0 -> 3 -> 1 -> 0 ")

Trip # 0 - Drone: 0 ( Trip- 1 ) Drop Sequence : ['01', '13', '30']
Trip # 1 - Drone: 1 ( Trip- 1 ) Drop Sequence : ['02', '20']
Trip # 2 - Drone: 2 ( Trip- 2 ) Drop Sequence : ['02', '20']
Trip # 3 - Drone: 0 ( Trip- 2 ) Drop Sequence : ['04', '41', '10']
Interpretation:
If output is ['03', '31', '10'] : It means it will move from 0 -> 3 -> 1 -> 0 


## **D. References**

# Drone Camera Fault Diagnoses

Now that your drone shipments are up and running with an optimized schedule, your team has begun using the each drone's sensors to capture additional data.  Each drone is equipped with an infrared (IR) camera and a visible spectrum camera. With these camera systems you have been able to make some extremely valuable observations of wildlife to include a possible sighting of Sasquatch (image shown below).

<img src=Bigfoot_Picture.png style="width: 60%;">

Unfortunately, you have encountered some issues with these camera systems that, in absence of an accurate diagnosis, require the drones to be sent back to your university to repair. This process takes several weeks. As a result, drone camera malfunctions are quite quite costly as you must choose to either (a) negatively affect the supply delivery schedule by sending the drone home for repairs, or (b) losing the opportunity to make observations during your supply runs or observation missions. 

You believe you are very close to finding a Sasquatch village, so you decide to take a look at the system schematic to see if there is a way you can diagnose malfunctions based on the system behavior. The schematic is shown below. In the image below, 

<img src=Logic_Array.png style="width: 60%;">

Each component is modular and easily accessible when the drone is in-hand and hence it can be repaire easily if you can correctly identify the problem. Unfortunately the drone's software only activates the camera systems when the drone is airborne so it is not possible to troubleshoot this issue by measuring voltages or some other method. Instead you (or maybe it occurs to you, an algorithm) must reason based on the system's behavior.  

In normal operation, the power relays provide power whenever they are supplied. The power conditioining units (PCUs) output synchronzied, conditioned power for the cameras whenever it is supplied with power from two power relays. The infrared and visibile spectrum cameras capture and store images whenever they are supplied with power from a PCU.

If a power relay fails, it will not provide power when supplied. A faulty PCU may not provide conditioned power even if both of its power relays are providing power. Additionally, a faulty PCU may provide "conditioned" power even if only one power relay is providing power (power supplied in such a state will lead to degraded images with either a faulty camera or a normal camera). However, a faulty PCU cannot provide power in absence of any supplied power. If a camera fails, it may not capture images even if provided with conditioned power.

According to the drone manufacturer, the power relays have a probability of failure of 1.5%, the PCUs have a probability of failure of 3% and the cameras have a probability of failure of 2.5%.

## System Model

You realized that once again, you have an opportunity to bring your knowledge from  6.877/16.413 to bear. You dvelope a system model using the variable definitions below:

1. A, B, and C: values are True if power is being supplied to power relays 1, 2, and 3, respectively.
2. D and E: values are True if images are saved by the IR camera and visible spectrum camera, respectively.
3. V, W, X: values are True or False depending on the behavior of power relays 1, 2, and 3, respectively, and the inputs A, B, and C, respectively.
4. Y and Z: values are True of False depending on the behavior of PCU1 and PCU2, respectively, and the values of V and W or W and X, respectively. 
5. P1, P2, and P3: values are True if power relays 1, 2, and 3, respectively, are behaving normally.
6. PCU1 and PCU2: values are True if PCU1 and PCU2, respectively, are behaving normally.
7. C1 and C2: values are True of the IR camera and visible spectrum camera, respectively, are behaving normally.

Based on the system description you develop the following sentences that, when conjoined, describe the system behavior. 

1. If power relay 1, is operating normally, then it relays power if it is supplied.

$$ \text{P1} \Longrightarrow \left( \text{V} \Longleftrightarrow \text{A} \right)$$

$$ \lnot \text{P1} \lor \left[ \left( \text{V} \Longrightarrow \text{A} \right) \land
\left( \text{A} \Longrightarrow \text{V} \right) \right)$$

$$ \lnot \text{P1} \lor \left[ \left( \lnot \text{V} \lor \text{A} \right) \land
\left( \lnot \text{A} \lor \text{V} \right) \right)$$

$$ \left( \lnot \text{P1} \lor \lnot \text{V} \lor \text{A} \right) \land
\left( \lnot \text{P1} \lor \lnot \text{A} \lor \text{V} \right)$$

2. If the power relay 1 fails, it will not provide power even if it is supplied.

$$ \lnot \text{P1} \Longrightarrow \lnot \text{V}$$

$$ \lnot \lnot \text{P1} \lor \lnot \text{V}$$

$$ \left( \text{P1} \lor \lnot \text{V} \right)$$


3. If power relay 2, is operating normally, then it relays power if it is supplied. By inspection, you know that this will reduce in the same fashion as constraint (1).

$$ \text{P2} \Longrightarrow \left( \text{W} \Longleftrightarrow \text{B} \right)$$

$$ \left( \lnot \text{P2} \lor  \lnot \text{W} \lor \text{B} \right) \land \left( \lnot \text{P2} \lor \lnot \text{B} \lor \text{W} \right)$$

4. If the power relay 2 fails, it will not provide power even if it is supplied.

$$ \lnot \text{P2} \Longrightarrow \lnot \text{W}$$

$$ \left( \text{P2} \lor \lnot \text{W} \right)$$

5. If power relay 3, is operating normally, then it relays power if it is supplied.

$$ \text{P3} \Longrightarrow \left( \text{X} \Longleftrightarrow \text{C} \right)$$

$$ \left( \lnot \text{P3} \lor  \lnot \text{X} \lor \text{C} \right) \land \left( \lnot \text{P3} \lor \lnot \text{C} \lor \text{X} \right)$$

6. If the power relay 3 fails, it will not provide power even if it is supplied.

$$ \lnot \text{P3} \Longrightarrow \lnot \text{X}$$

$$ \left( \text{P3} \lor \lnot \text{X} \right)$$

7. If PCU 1 is operating normally, then it provides conditioned power only if supplied by two sources.

$$ \text{PCU1} \Longrightarrow \left( \text{Y} \Longleftrightarrow \left( \text{V} \land \text{W} \right) \right)$$

$$ \lnot \text{PCU1} \lor \left[ \left( \text{Y} \Longrightarrow \left( \text{V} \land \text{W} \right) \right) \land \left( \left( \text{V} \land \text{W} \right) \Longrightarrow \text{Y} \right) \right]$$

$$ \lnot \text{PCU1} \lor \left[ \left( \lnot \text{Y} \lor \left( \text{V} \land \text{W} \right) \right) \land \left( \lnot \left( \text{V} \land \text{W} \right) \lor \text{Y} \right) \right]$$

$$ \lnot \text{PCU1} \lor \left[ \left( \left( \lnot \text{Y} \lor \text{V} \right) \land \left( \lnot \text{Y} \lor \text{W} \right) \right) \land \left( \lnot \text{V} \lor \lnot \text{W} \lor \text{Y} \right) \right]$$

$$ \left( \lnot \text{PCU1} \lor \lnot \text{Y} \lor \text{V} \right) \land \left( \lnot \text{PCU1} \lor \lnot \text{Y} \lor \text{W} \right) \land \left( \lnot \text{PCU1} \lor \lnot \text{V} \lor \lnot \text{W} \lor \text{Y} \right)$$

8. If PCU 1 has failed, then it may or may not supply power so long as it has at least one power source.

$$ \left( \lnot \text{PCU1} \land \text{Y} \right) \Longrightarrow \left( \text{V} \lor \text{W} \right)$$

$$ \lnot \left( \lnot \text{PCU1} \land \text{Y} \right) \lor \left( \text{V} \lor \text{W} \right)$$

$$  \left( \text{PCU1} \lor \lnot \text{Y} \right) \lor \left( \text{V} \lor \text{W} \right)$$


$$  \left( \text{PCU1} \lor \lnot \text{Y} \lor \text{V} \lor \text{W} \right)$$

9. If PCU 2 is operating normally, then it provides conditioned power only if supplied by two sources.

$$ \text{PCU2} \Longrightarrow \left( \text{Z} \Longleftrightarrow \left( \text{W} \land \text{X} \right) \right)$$

$$ \left( \lnot \text{PCU2} \lor \lnot \text{Z} \lor \text{W} \right) \land \left( \lnot \text{PCU1} \lor \lnot \text{Z} \lor \text{X} \right) \land \left( \lnot \text{PCU1} \lor \lnot \text{W} \lor \lnot \text{X} \lor \text{Z} \right)$$
10. If PCU 2 has failed, then it may or may not supply power so long as it has at least one power source.

$$ \left( \lnot \text{PCU2} \land \text{Z} \right) \Longrightarrow \left( \text{W} \lor \text{X} \right)$$

$$  \left( \text{PCU2} \lor \lnot \text{Z} \lor \text{W} \lor \text{X} \right)$$

11. If the IR camera is operating normally, then it will save good images.

$$ \text{C1} \Longrightarrow \left( \text{Y} \Longleftrightarrow \text{D} \right)$$

$$ \left( \lnot \text{C1} \lor  \lnot \text{Y} \lor \text{D} \right) \land \left( \lnot \text{C1} \lor \lnot \text{D} \lor \text{Y} \right)$$

12. If the visible spectrum camera is operating normally, then it wwill save good images.
$$ \text{C2} \Longrightarrow \left( \text{Z} \Longleftrightarrow \text{E} \right)$$

$$ \left( \lnot \text{C2} \lor  \lnot \text{Z} \lor \text{E} \right) \land \left( \lnot \text{C2} \lor \lnot \text{E} \lor \text{Z} \right)$$

## Conflict-Directed A*
Based on this model and the associated probabilities, you decide to implement conflict-directed A*. To implement this, you know you need to take a set of observations (specified by the states of D and E), inputs (specified by the states of A, B, and C) and use these to generate conflicts to search through the tree.  Moreoever, you know you should start your search with the moste likely system configuration--namely, that all components are working normally.

Using this as a starting-point, conflict-directed A* will leverage De Morgan's Theorem to generate kernels that *could* explain the system behavior. It will then iteratively select the best kernel (most likely kernel) and assign the remaining unassigned components to their most likely state. If it observes a conflict, it will continue with this process recursively.

With this approach, the most likely conflicts will be generated in order of probability for a given behavior.


This algorithm is implemented below.

'''These types will be used to encode the clauses and propositions:

Model (variable)
-   A set of clauses

Visited list (variable)
-	A set of visited candidates
-	Each visited candidate is a set
-   As build the candidates from the kernels, check to see if it has already been visited

Visited kernels (variable)
-	A set of visited kernels
-	Each visited kernel
-   As build the candidates from the kernels, check to see if it has already been visited


Clause (a class)
-	Propositions - set (attribute)
    o	Set
    o	Each element is a proposition data type
-	Not operators or assignements- set (attribute)
    o	Set
    o	5 indicates absence of a not
    o	-1 indicates a not
    o	Alternatively, this set can also contain specify the value options e.g. '0', '1', '2'
        This would indicated if the clause asserts that the component in question is in mode 0, 1, or 2
-	The propositions in the Clause class are assumed to be joined by or’s

Proposition (a class)
-	Name (attribute)
   o	Component name, e.g. ‘P1’
-	Domain (attribute)
   o	A set  of possible mode tuples, where the first element is the mode name and the second is the prior probability
   o    e.g {('0', 0.9), ('1', 0.18), ('2', 0.02)}
-   Domain (attribute)--alternative implementation
   o	A set  of possible modes,
   o    e.g {0, 1, 2}
-	Prob (attribute)--alternative implementation
   o	A set of probabilities associated with each mode, e.g. {0.9, 0.18, 0.02}
-	Cand_assignment (attribute)
   o	A single element list, e.g. 1
-	Cand_support (attribute)
   o	A single element list specifying index of the clause (index within model set), e.g. 0
   
Conflict (variable) – how to specifiy
-	conflict = set([('A1',1),('A2',1),('X1',1)]), where the first element in the tuple is the component and the second element is the mode
-	This will be driven by the clause from which it was derived using the information in the Cand_assignment 

kernel_diagnoses (variable)
-   A list of sets representing the currently held kernel diagnoses.

kernel_probabilities (variable)
-   A list specifying the unnormalized prior probability associated with each kernel in kernel diagnoses

compute_kernel_diagnoses
-   A function to compute the unnormalized prior probability of a kernel

'''

In [ ]:
def update_kernel_diagnoses(kernel_diagnoses, conflict):
    # create output variable
    output_kernel_diagnoses = []
    
    # function to invert element from a conflict into a diagnosis
    def return_diagnosis(elem):
        diagnosis = ()
        if elem[1] == 1:
            diagnosis = (elem[0],0)
        else:
            diagnosis = (elem[0],1)
        return diagnosis
    
    # Convert the conflict set (A and B and...) to a set of candidate_diagnoses (not A or not B or...) (application of DeMorgan's Theorem)
    candidate_diagnoses = set()
    for elem in conflict:
        candidate_diagnoses.add(return_diagnosis(elem))
        
    
    # 0. Check to see if kernel_diagnoses is empty, if so add each element of candidate_diagnoses to the output
    if len(kernel_diagnoses) == 0:
        for elem in candidate_diagnoses:
            output_kernel_diagnoses.append(set([elem]))
        
    else:
    # 1. For all kernels in kernel_diagnoses, check to see if is a subset of candidate_diagnoses.
        # If it is, then remove it from the diagnoses, remove it from conflict, and add it to the output
        # Removal is necessary so don't form supersets in step #2
        elim_list = []
        for kernel in kernel_diagnoses:
            if kernel.issubset(candidate_diagnoses):
                elim_list.append(kernel)    # Track for removal from the diagnoses set
                for elem in kernel:
                    candidate_diagnoses.remove(elem)
                output_kernel_diagnoses.append(kernel)
        
        for kernel in elim_list:
            kernel_diagnoses.remove(kernel)
        
    #2. Add remaining elements in conflict to the remaining kernels in dummy_kernel_diagnoses to form new kernels
        for rem_elem in candidate_diagnoses:
            for rem_kernel in kernel_diagnoses:
                    # remove rem_kernal from the diagnoses and add item to it
                    addition = set()
                    addition.add(rem_elem)
                    for elem in rem_kernel:
                        addition.add(elem)
                    output_kernel_diagnoses.append(addition)
    return output_kernel_diagnoses

def conflict_directed_a_star(model, components, mode_probs, inputs, observations):
    '''model is a dictionary encoding the constraints relevant for each component:
            model = {component1: [['componenent1: True, component 2: False'...],...]}
    
       components is a dictionary of the form:
            components = {component_1: [modeA, modeB, ...],
                          component_2: [modeA, modeB, ...],
                          ...}
       mode_probs is a dictionary of the form:
            mode_probs = {component1: [P(modeA), P(modeB)...],
                          component2: [P(modeA), P(modeB)...],
                          ...}
        inputs is a dictionary of the form:
            inputs = {input_1: mode,
                          input_2: mode,
                          ...}
        observations is a dictionary of the form:
            observations = {input_1: mode,
                          input_2: mode,
                          ...}
        prob_ordered_dictionary is the output. The key is an index ordered by prior probability. The vlaues are 
        a set where each element of that set is as complete mode assignment.
        The first element of each set is the posterior probability of that assignment
        
        Will have helper functions to:
            update_kernel_diagnoses
        '''